In [1]:
# to load the fine tuned model
from langchain.llms import CTransformers 

In [2]:
#Loading the model from local storage and adjusting the necessary parameter . 
#temperature parameter ranges from 0 to 1 where lower value means less creative with straightforward response where higher values means more creative but chances of response being inaccurate
# max_new_tokens is context window for llm (i.e total tokens llm can accept at a time) 
#context_length is the number of tokens  that can be generated as response by llm
llm = CTransformers(model="model\\nous-research-finetuned-1epoch.Q4_K_M.gguf",
                        model_type="llama", config={'max_new_tokens':5990,'temperature':0.6,'context_length': 5990})

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE transactions (
	`Account_No` VARCHAR(50) NOT NULL, 
	`Transaction_details` TEXT, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_amount` INTEGER, 
	`Value_date` DATE, 
	`Date` DATE
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from transactions table:
Account_No	Transaction_details	Withdrawal_amount	Deposit_amount	Balance_amount	Value_date	Date
409000611074	TRF FROM  Indiaforensic SERVICES	0	1000000	1000000	2022-11-16	2022-11-16
409000611074	TRF FROM  Indiaforensic SERVICES	0	1000000	2000000	2022-11-22	2022-11-22
409000611074	FDRL/INTERNAL FUND TRANSFE	0	500000	2500000	2022-12-05	2022-12-05
*/


In [4]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)


# FEW SHORT LEARNING

In [5]:
#Few_shorts

#This few shots example consists of more questions that the model was not trained on and the model uses these as references.

few_shots=[
    {
        'Question':" Income of last 3 months. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"1530205868",
        'Answer':"The income of last 3 months is 1530205868 ."
    }  
,
    {
        'Question':"How many transactions did I make in last 2 week as my account number is 409000493201? ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK) AND ACCOUNT_NO= '409000493201' """,
        'SQLResult':"162",
        'Answer':"You made 162 transactions in last 2 week."
    }
    ,
     {
        'Question':"Amount spent last week as my account number is 409000493201?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE year(value_date) = year(curdate()) and week(value_date)=week(curdate())-1  AND ACCOUNT_NO= '409000493201' """,
        'SQLResult':"626764",
        'Answer':"You spent total RS. 626764 last week ."
    }
    , {
        'Question': "Expenses for each day this month ?",
        'SQLQuery': """SELECT value_date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY value_date """,
        'SQLResult':
        """
[(2024, 6, 2, Decimal('183648')),
(2024, 6, 3, Decimal('80666')),
(2024, 6, 4, Decimal('135504')),
(2024, 6, 5, Decimal('92031')),
(2024, 6, 6, Decimal('118475')),
(2024, 6, 7, Decimal('16440')),
(2024, 6, 9, Decimal('62112')),
(2024, 6, 10, Decimal('10331'))]
"""
        ,
        'Answer': """The total_expenses for each day of this month are:
        2024 June, 2:RS. 183648,
        2024 June, 3:RS. 80666,
        2024 June, 4:RS. 135504,
        2024 June, 5:RS. 92031,
        2024 June, 6:RS. 118475,
        2024 June, 7:RS. 16440,
        2024 June, 9:RS. 62112,
        2024 June, 10:RS. 10331
        """
    },
    {
        'Question':"savings of previous year for each month ",
        'SQLQuery':"""SELECT year(Value_date), MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY year(value_date), MONTH(Value_date) ORDER BY MONTH(Value_date); """,
        'SQLResult':"""
       [(2023, 1, Decimal('-311244')),
        (2023, 2, Decimal('9101')),
        (2023, 3, Decimal('3476')),
        (2023, 4, Decimal('-15971')),
        (2023, 5, Decimal('322249')),
        (2023, 6, Decimal('-172560')),
        (2023, 7, Decimal('361471')),
        (2023, 8, Decimal('-446844')),
        (2023, 9, Decimal('93215')),
        (2023, 10, Decimal('-73917')),
        (2023, 11, Decimal('294502')),
        (2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""
        Your Savings for each month of previous year are:
        January:RS. -311244, February:RS. 9101, March:RS. 3476
        April:RS. -15971, May:RS. 322249, June:RS. -172560
        July:RS. 361471, August:RS. -446844, September:RS. 93215
        October:RS. -73917, November:RS. 294502, December:RS. -404028 """
    },
     {
        'Question':"Amount Saved each year",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
      [(2021, Decimal('497766')),
        (2022, Decimal('272458')),
        (2023, Decimal('-340550')),
        (2024, Decimal('-148114'))]
       """,
        'Answer':"""You saved following amount each year:
                2021:RS. 497766, 2022:RS. 272458, 2023:RS. -340550
                2024:RS. -148114 """
    }
    ,
     {
        'Question':"Savings of last 2 year as my account number is 409000493201? ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) AND ACCOUNT_NO= '409000493201' GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
       [(2022, Decimal('2270453')),
    (2023, Decimal('-895783')),
    (2024, Decimal('-912470'))]

        """,
        'Answer':"""Your savings of last 2 year are:
         2022:RS. 2270453, 2023:RS.-895783 
                2024:RS. -912470 ."""
    }

    #&&&&&&&&&&&&&&&&&&&&
    ,
     {
        'Question':"Expenses of each day of last week for account number 409000438611  ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE  YEAR(Value_date) = YEAR(CURDATE()) AND week(Value_date) = week(CURDATE())-1 and account_no='409000438611' GROUP BY DATE(Value_date)""",
        'SQLResult':"""
       [(2024-07-14, Decimal('12500000')),
    (2024-07-15, Decimal('4900000')),
    (2024-07-16, Decimal('3901373')),
    (2024-07-17, Decimal('4400000')),
    (2024-07-18, Decimal('4800000')),
    (2024-07-19, Decimal('2700000'))]
        """,
        'Answer':"""You spend following amount each day last week:
        July 14:RS. 12500000, July 15:RS. 3901373, July 16:RS. 3901373, July 17:RS. 4400000, July 18:RS. 4800000, July 19:RS. 2700000 """
    }
    ,
    {
        'Question':"Amount spent this year as my account number is 409000493201??",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE year(Value_date) = year(curdate()) AND ACCOUNT_NO= '409000493201' group by year(Value_date) """,
        'SQLResult':"""[(2024, Decimal('6527128485'))]""",
        'Answer':"You spent RS. 6527128485 on 2024"
    },
    {
    'Question': "What is my consistent income in last 2 months as my account number is 409000493201  ?",
    'SQLQuery': """SELECT AVG(Monthly_Income) AS Consistent_Income 
    FROM (SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income FROM transactions WHERE  Value_date >= 
    DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH) AND ACCOUNT_NO= '409000493201'
    GROUP BY  YEAR(Value_date), MONTH(Value_date)) AS Income_Last_3_Months;
    """,
    'SQLResult': "2466666.6667",
    'Answer': "The consistent income of last 2 months is RS. 2466666.6667"
}
    ,
    {
        'Question':"amount/money saved each month previous year ?",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR)) GROUP BY YEAR(Value_date), MONTH(Value_date) """,
        'SQLResult':"""
[(2023, 1, Decimal('-311244')),
(2023, 2, Decimal('9101')),
(2023, 3, Decimal('3476')),
(2023, 4, Decimal('-15971')),
(2023, 5, Decimal('322249')),
(2023, 6, Decimal('-172560')),
(2023, 7, Decimal('361471')),
(2023, 8, Decimal('-446844')),
(2023, 9, Decimal('93215')),
(2023, 10, Decimal('-73917')),
(2023, 11, Decimal('294502')),
(2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""You saved following amounts previous year :
        January: -311244, February: 9101, March: 3476, April: -15971, May: 322249, June: 172560, July: 361471, 
        August: 446844, September: 93215, October: 73917, November: 294502, December: 404028 """
    }
    ,
{
    'Question': "What is my total income last quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1""",
    'SQLResult':"2055417596",
    'Answer':"Your total income last quater was RS. 2055417596."
}
     ,
      {
        'Question':"income of the previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 """,
        'SQLResult':"38593214103",
        'Answer':"Your income previous ysear was 38593214103."
    }
    ,
      {
        'Question':"income of the first 4 month of previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"17847505444",
        'Answer':"Your income of first 4 month of previous year was 17847505444."
    }
    ,
    {
    'Question': "amount I lost each year?",
    'SQLQuery': """ SELECT YEAR(Value_date) AS Year,SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Losses FROM transactions GROUP BY YEAR(Value_date) """,
    'SQLResult': """[
        (2022, Decimal('256395842')),
        (2023, Decimal('552520828')),
        (2024, Decimal('505236609')),
        (2021, Decimal('1251859488')),
        (2020, Decimal('694854538'))]"""
    ,
    'Answer': "You lost RS. 694854538 in 2020, RS. 1251859488 in 2021, RS. 256395842 in 2022,RS. 552520828 in 2023, and RS. 505236609 in 2024."
}
,
{
    "Question": "What is my average spendings of last 11 transactions?",
    "SQLQuery": """SELECT AVG(Withdrawal_amount) AS Average_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount 
                IS NOT NULL ORDER BY Value_date DESC LIMIT 11) AS Last11Transactions""",
    "SQLResult":"1138377.5455",
    'Answer': "1138377.5455 is your average spendings of last 11 transactions ." 
}
,
{
    'Question': "How much did I spend each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY QUARTER(Value_date)""",
    'SQLResult':"""[(1, Decimal('5885524375')),
                    (2, Decimal('2689194690')),
                    (3, Decimal('263256582'))]""",
    'Answer': """Spendings for each quater is :
                    Quarter-1:RS. 5885524375, Quarter-2:RS. 2689194690, Quarter-3:RS. 263256582"""
},

{
    'Question': "In which date did i make my highest transaction of this month and what was the amount?",
    'SQLQuery': "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1",
    'SQLResult': "[(2024-06-25, Decimal('10000000'))]",
    'Answer': "You made the highest transaction on Junt 25th and the amount is RS. 10000000"
}
,
 {
        'Question': "How much amount did I spent on February month ?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Spending_February FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = 2""",
        'SQLResult': "1985783353",
        'Answer': "You spent RS. 1985783353 on February month."
    },
    {
        'Question': "What were my total expenditures by month in the previous year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY YEAR(Value_date), MONTH(Value_date)""",
        'SQLResult':"""[(2023, 1, Decimal('4530730974')), (2023, 2, Decimal('4616646063')), (2023, 3, Decimal('5276664685')), (2023, 4, Decimal('4571039249')), (2023, 5, Decimal('3557364039')), (2023, 6, Decimal('3262112460')), (2023, 7, Decimal('3142750834')), (2023, 8, Decimal('2875423837')), (2023, 9, Decimal('2727745395')), (2023, 10, Decimal('2537575257')), (2023, 11, Decimal('2818821756')), (2023, 12, Decimal('2246381750'))]""",
        'Answer': """Expenditure for each month previous year:
            January:RS. 4530730974, February: RS. 4616646063, March: RS. 5276664685, April: RS. 4571039249, May:RS. 3557364039, June: RS. 3262112460, July:RS. 3142750834, August:RS. 2875423837, September:RS. 2727745395, October:RS. 2537575257, November:RS. 2818821756, December:RS. 2246381750 """
}

,{
        'Question': "Which is my highest earning month this year ? I also want the amount along with month",
        'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY MONTH(Value_date) ORDER BY Total_Earnings DESC limit 1""",
    'SQLResult': "[(1, Decimal('1893013978'))]",
        'Answer': "Your highest earning month this year is June with amount 1893013978."
    }
    ,{
        'Question': "What was my total spending last week? as my account number is 409000438611 ",
        'SQLQuery': """SELECT sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE  YEAR(Value_date) = YEAR(CURDATE()) AND week(Value_date) = week(CURDATE())-1  
        AND ACCOUNT_NO= '409000438611'""",
        'SQLResult': "33201373",
        'Answer': "Your total expenses last week was RS. 33201373."
    },
#     {
#     "Question": "Can you give me a breakdown of my expenses for each day this week?",
#     "SQLQuery": "SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND WEEK(Value_date) = WEEK(CURDATE()) GROUP BY DATE(Value_date) ORDER BY Date ASC",
#     'SQLResult':"""(2024-07-21, Decimal('9967448')),
#         (2024-07-22, Decimal(32142360'))""",
#     'Answer': "Daily expenses for the week: July 21st: RS. 9967448, July 22nd: RS. 32142360."
# }
# {
#         'Question': "How much did i save last month as my account number is 409000493201 ",
#         'SQLQuery': """  SELECT  SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
# FROM transactions WHERE year(value_date) = year(curdate()) and month(value_date)=month(curdate()) -1 AND Account_No = '409000493201';
# """,
#     'SQLResult': "89804",
#         'Answer': "Your savings last month was RS. 89804."
#     }
#     ,
    {
        'Question': "What was my total income and total spending last month? as my account number is 409000425051 ",
        'SQLQuery': """  SELECT SUM(Deposit_amount) AS Total_Income, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1 AND Account_No = '409000493201'; """,
    'SQLResult': "[(Decimal('4700000'), Decimal('4610196'))]",
        'Answer': "Your total income last month was RS. 4700000 and total spending was RS. 4610196 ."
    },
    {
    'Question': "Breakdown my expenses for each week for the last month as my account number is 409000493201.",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Withdrawal_amount) AS Weekly_Expenses 
                   FROM transactions 
                   WHERE Account_No = '409000493201' 
                   AND Value_date >= DATE_SUB(LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH)), INTERVAL DAY(LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH)))-1 DAY)
                   AND Value_date <= LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH))
                   GROUP BY 1 """,
    'SQLResult': """[(22, Decimal('1113224')),
                    (23, Decimal('1030576')),
                    (23, Decimal('1394682')),
                    (24, Decimal('847515')),
                    (25, Decimal('224199'))]""",
    'Answer': """ Your expenses were:
    1st week : RS. 1113224,
    2nd week :RS. 1030576,
    3rd week: RS. 1394682,
    4th week:RS. 847515 , 
    5th week:RS. 224199"""
  
}
,
{
        'Question': "What is my total income for this year so far as my account number is 409000611074?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income 
FROM transactions 
WHERE YEAR(Value_date) = YEAR(CURDATE()) 
AND Account_No = '409000611074';
 """,
    'SQLResult': "59000000",
        'Answer': "The total income this year is RS. 59000000 "
    },
    {
        'Question': "How much did I spend in the last quarter of the previous year? as my account number is 409000611074?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending 
FROM transactions 
WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1 
AND Account_No = '409000611074'
AND QUARTER(Value_date) = 4; """,
    'SQLResult': "32120960",
        'Answer': "Your spendings of last quater is RS.32120960 "
    },
    {
        'Question':  "Can you give me a monthly breakdown of my income for the past 6 months? as my account number is 409000493201",
        'SQLQuery': """ SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income 
FROM transactions 
WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH) 
AND Account_No = '409000493201'
GROUP BY YEAR(Value_date), MONTH(Value_date);""",
    'SQLResult': """[(2024,1, Decimal('715160')),
                    (2024,2, Decimal('4584217')),
                    (2024,3, Decimal('9112141')),
                    (2024,4, Decimal('11443808')),
                    (2024,5, Decimal('5411781')),
                    (2024,6, Decimal('4700000')),
                    (2024,7, Decimal('2100000'))]""",
        'Answer': """Here is the breakdown of your monthly income for past 6 months:
        January: RS. 715160,
        February: RS. 4584217,
        March: RS. 9112141,
        April: RS. 11443808,
        May: RS. 5411781,
        June: RS. 4700000,
        July: RS. 2100000
        """
    },
    {
        'Question': "What is the total amount of money deposited over the past year for account number 409000493201?",
        'SQLQuery': """ SELECT SUM(Deposit_amount) AS Total_Deposits 
FROM transactions 
WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1 
AND Account_No = '409000493201'""",
    'SQLResult': "39109096",
        'Answer': "The total amount of money deposited over the past year is Rs. 39109096"
    },
    #$$$$$$

{
    'Question': "What were my total losses for each month last year for account number 409000493201?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Monthly_Losses 
                   FROM transactions 
                   WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1 
                   AND Account_No = '409000493201'
                   GROUP BY MONTH(Value_date);""",
    'SQLResult': """[(1, Decimal('463794')),
                    (2, Decimal('-5936')),
                    (3, Decimal('-262405')),
                    (4, Decimal('403300')),
                    (5, Decimal('-170150')),
                    (6, Decimal('-363342')),
                    (7, Decimal('456354')),
                    (8, Decimal('-486051')),
                    (9, Decimal('267825')),
                    (10, Decimal('-179758')),
                    (11, Decimal('74787')),
                    (12, Decimal('-122188'))]""",
    'Answer': """Your loss for each month last year is:
    January: RS. 463794,
     February:RS.-5936,
     March:RS. -262405,
     April:RS. 403300,
     May:RS. -170150,
     June:RS. -363342,
     July:RS. 456354,
     August:RS. -486051,
     September:RS. 267825,
     October:RS. -179758,
     November:RS. 74787,
     December:RS. -122188"""
},
{
    'Question': "What is the highest single deposit amount made this year for account number 409000493201?",
    'SQLQuery': """SELECT MAX(Deposit_amount) AS Highest_Deposit 
                   FROM transactions 
                   WHERE YEAR(Value_date) = YEAR(CURDATE()) 
                   AND Account_No = '409000493201';""",
    'SQLResult': "615753",
    'Answer': "The highest deposit made this year is RS. 615753"
},
{
    'Question': "How much did I spend on the 1st and 2nd of this month for account number 409000493201?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending 
                   FROM transactions 
                   WHERE YEAR(Value_date) = YEAR(CURDATE()) 
                   AND MONTH(Value_date) = MONTH(CURDATE()) 
                   AND DAY(Value_date) IN (1, 2) 
                   AND Account_No = '409000493201'
                   GROUP BY DATE(Value_date);""",
    'SQLResult': """[(2024-07-01, Decimal('116967')),
                    (2024-07-02, Decimal('90749'))""",
    'Answer': """You spent following amount :
    07-01: RS. 116967,
     07-02: RS. 90749 """
},
{
    'Question': "Can you show my total savings for each quarter of the current year for account number 409000493201?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Quarterly_Savings 
                   FROM transactions 
                   WHERE YEAR(Value_date) = YEAR(CURDATE()) 
                   AND Account_No = '409000493201'
                   GROUP BY QUARTER(Value_date);""",
    'SQLResult': """[(1, Decimal('-566948')),
                    (2, Decimal('-74404')),
                    (3, Decimal('44031'))]""",
    'Answer': "1st quater : you lossed RS. -566948 ,2nd quater : you lossed RS. 74404 ,3rd quater : you earned RS 44031"
},
#From below down the model has not been fine tuned with the question
{
    'Question': "I want to buy a car worth RS. 5,00,000 in next 2 years . So how much should i save each month to buy the car  for account number 409000493201?",
    'SQLQuery': """SELECT round(AVG(Monthly_Savings),2) AS Average_Monthly_Savings
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, 
           SUM(Deposit_amount - Withdrawal_amount) AS Monthly_Savings
    FROM transactions
    WHERE Account_No = '409000493201'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Monthly_Savings_Calculation;""",
    'SQLResult': """9385.33""",
    'Answer': """Lets think step by step:
        step-1) Determine Total Number of Months:
        Time frame: 2 years (12 months in 1 year So 2 years * 12 months = 24) = 24 months
    step-2) Calculate Required Monthly Savings:
        Target amount: RS 500000
        Required monthly savings:(It is obtained by dividing the total amount by Time frame) i.e RS. 500000 / 24 months = RS. 20,833.33
    step-3) Compare Current Savings with Required Savings:
        Current average monthly savings: RS 9385.33
        Difference needed:  RS. 20,833.33 - RS 9385.33 = RS. 11,448
    step-4) Conclusion: To reach the goal of saving RS 5,00,000 in 2 years. increase monthly savings to RS 20,833.33 .The additional amount to save each month is RS. 11,448 """
}
,
{
    'Question': "I want to buy a car worth RS 6,00,000 in next 3 years . So how much should i save each month to buy the car ? as my account number is 409000438611. ",
    'SQLQuery': """SELECT AVG(Monthly_Savings) AS Average_Monthly_Savings
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, 
           SUM(Deposit_amount - Withdrawal_amount) AS Monthly_Savings
    FROM transactions
    WHERE Account_No = '409000438611'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Monthly_Savings_Calculation;""",
    'SQLResult': """9627.44""",
    'Answer': """Lets think step by step:
        step-1) Determine Total Number of Months:
        Time frame: 3 years (12 months in 1 year So 3 years * 12 months = 36) = 36 months

    step-2) Calculate Required Monthly Savings:
        Target amount: RS 6,00,000
        Required monthly savings:(It is obtained by dividing the total amount by Time frame)RS 600000 / 36 months = RS 16,666.67

    step-3) Compare Current Savings with Required Savings:
        Current average monthly savings: RS 9627.44
        Difference needed: RS 16,666.67 - RS 9,627.44 = RS 7,039.23

    step-4) Conclusion: To reach the goal of saving RS 6,00,000 in 3 years, increase monthly savings to RS 16,666.67 The additional amount to save each month is RS 7,039.23"""
}
,

{
    'Question': "What is my average monthly income in first 6 month this year? as my account number is 409000493201. ",
    'SQLQuery': """SELECT AVG(Monthly_Income) AS Average_Income 
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income 
    FROM transactions 
    WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) 
      AND MONTH(Value_date) BETWEEN 1 AND 6
      AND Account_No = '409000493201'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Income_First_6_Months;
""",
    'SQLResult': """6352764.1667 """,
    'Answer': """Your Average monthly income of the first 6 month this year is RS. 6352764.1667"""
    },
    {
    "Question": "How many transactions did I make last quarter ? as my account number is 409000493201.",
    "SQLQuery": "SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1 AND Account_No = '409000493201';",
    'SQLResult': """145""",
    'Answer': """You made 145 transactions last quater."""
}
,
{
    'Question': "What is my total number of transactions each month this year?",
    'SQLQuery':""" SELECT MONTH(Value_date) AS Month, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date);""",
    'SQLResult': """[(1, 3567),
 (2, 3468),
 (3, 2752),
 (4, 2729),
 (5, 2603),
 (6, 2653),
 (7, 1923)]""",
'Answer':""" You made following number of transactions each month this year:
January: 3567, February: 3468, March: 2752, April: 2729, May: 2603, June: 2653, July: 1923"""
}
,
{
    "Question": "What is my net worth as of today?",
    "SQLQuery": "SELECT Balance_amount AS Net_Worth FROM transactions WHERE Account_No = '409000493201' ORDER BY Value_date DESC LIMIT 1;",
    'SQLResult': """743583 """,
    'Answer': """Your net worth as of today is RS. 743583"""
}
,
{
    'Question': "How many withdrawals did I make this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Withdrawal_amount IS NOT NULL""",
    'SQLResult': """19695 """,
    'Answer': """Your made 19695 transaction this year."""
},
{
    'Question': "What is the average daily spending in the last month?",
    'SQLQuery': """SELECT AVG(Daily_Spending) AS Average_Daily_Spending
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
                       GROUP BY DATE(Value_date)
                   ) AS DailySpending""",
    'SQLResult': """18662884.0417 """,
    'Answer': """Your average daily spending last month was RS. 18662884.0417"""
},
{
    'Question': "What is my largest withdrawal amount this year? as my account number is 409000425051",
    'SQLQuery': """SELECT MAX(Withdrawal_amount) AS Largest_Withdrawal
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   And Account_no='409000425051'""",
    'SQLResult': """354000000 """,
    'Answer': """Your largest withdrawal amount this year is RS. 354000000"""
}



]



In [6]:
pwd

'c:\\Users\\Prabal Kuinkel\\Desktop\\ARL-Intern\\privategpt'

In [7]:
import os

# Define the path to save the vector store
CHROMA_DIR = "vectorstore/chroma"
CHROMA_DB_PATH = os.path.join(CHROMA_DIR, "index")

In [8]:
try:
    # Ensure the vectorstore directory exists
    os.makedirs(CHROMA_DIR, exist_ok=True)
    print(f"Directory created or already exists: {CHROMA_DIR}")
except FileNotFoundError as e:
    print(f"Error creating directory: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Directory created or already exists: vectorstore/chroma


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# Check if the vector store already exists
if os.path.exists(CHROMA_DB_PATH):
    print("Loading embeddings from existing Chroma vector store.")
    vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
else:
    print("Creating new embeddings and saving to Chroma vector store.")
    # Create embeddings and vector store
    to_vectorize = [" ".join(example.values()) for example in few_shots]
    vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots, persist_directory=CHROMA_DB_PATH)
    vectorstore.persist()
    print(f"New embeddings created and saved at: {CHROMA_DB_PATH}")

Creating new embeddings and saving to Chroma vector store.
New embeddings created and saved at: vectorstore/chroma\index


c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [11]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# give me the Breakdown of income for each day this week as my account number is 409000493201'

In [12]:
# Test the example selector with a query
selected_examples = example_selector.select_examples(
    {"Question": "what was my savings last month ?"}
)

In [13]:
selected_examples

[{'Answer': 'Your total income last month was RS. 4700000 and total spending was RS. 4610196 .',
  'Question': 'What was my total income and total spending last month? as my account number is 409000425051 ',
  'SQLQuery': "  SELECT SUM(Deposit_amount) AS Total_Income, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1 AND Account_No = '409000493201'; ",
  'SQLResult': "[(Decimal('4700000'), Decimal('4610196'))]"},
 {'Answer': 'You spent following amount :\n    07-01: RS. 116967,\n     07-02: RS. 90749 ',
  'Question': 'How much did I spend on the 1st and 2nd of this month for account number 409000493201?',
  'SQLQuery': "SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending \n                   FROM transactions \n                   WHERE YEAR(Value_date) = YEAR(CURDATE()) \n                   AND MONTH(Value_date) = MONTH(CURDATE()) \n                   AND DAY(V

In [14]:
# # Define the custom prompt for SQL database interactions
# custom_mysql_prompt = """
# You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question. Prepare the correct MySQL query by looking at the top {top_k} similar queries.
# Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
# Never query for all columns from a table. You must query only the columns that are needed to answer the question.
# Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. 
# Pay attention to use CURDATE() function to get the current date, if the question involves "today","this year","this month" ,"this week".

# Table information:
# 1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

# Remember to only use the provided table columns and structure your queries to handle the specified requests accurately.Only answer the question given by the user and dont assume any question by yourself.
#  """

In [15]:
# # Define the custom prompt for SQL database interactions
# custom_mysql_prompt = """You are an expert in converting natural language into MySQL query.Only use the columns that are specified and needed to answer the question.
# Use query for at most {top_k} results.
# Table information:
# 1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`
# """

In [16]:
mysql_prompt = """
You are an expert in converting natural language questions into MySQL queries. Your task is to generate a syntactically correct MySQL query based on the given input question, execute the query, and then provide an answer based on the query results.
Guidelines:
1. Construct a MySQL query to fetch data relevant to the user's question. Ensure the query is syntactically correct.
2. If the question does not specify the number of examples, use the LIMIT clause to restrict the results to at most {top_k} entries. Prioritize ordering the data to show the most informative entries.
3. Query only the necessary columns needed to answer the question. Avoid selecting all columns from a table.
4. Only use the column names specified in the table schema below. Do not reference any non-existent columns. Ensure each column name corresponds to the correct table.
5. Use the CURDATE() function for questions related to "today," "this year," "this month," or "this week."
6. Base your answer solely on the data returned in the SQLResult. Avoid assumptions beyond the provided data.
7. Answer only the user's question without introducing new questions or information.

Table information:
1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

Format:
Question: ["The user's question"]
SQLQuery: ["Your MySQL query without any preamble"]
SQLResult: ["The result of the MySQL query"]
Answer: ["The answer based on the SQLResult"]
"""


In [17]:
# Guidelines:
# 1. Construct a MySQL query to fetch data relevant to the user's question. Ensure the query is syntactically correct.
# 2. If the question does not specify the number of examples, use the LIMIT clause to restrict the results to at most {top_k} entries. Prioritize ordering the data to show the most informative entries.
# 3. Query only the necessary columns needed to answer the question. Avoid selecting all columns from a table.
# 4. Only use the column names specified in the table schema below. Do not reference any non-existent columns. Ensure each column name corresponds to the correct table.
# 5. Use the CURDATE() function for questions related to "today," "this year," "this month," or "this week."
# 6. Base your answer solely on the data returned in the SQLResult. Avoid assumptions beyond the provided data.
# 7. Answer only the user's question without introducing new questions or information.


In [18]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt

In [19]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [20]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [21]:
from langchain.prompts.prompt import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [22]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [23]:
suffix_eg="Question: {input}"

In [24]:
print(suffix_eg)

Question: {input}


In [25]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix="Question: {input}",
    input_variables=["input", "top_k"], #These variables are used in the prefix and suffix
)

In [26]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [27]:
print(few_shot_prompt)

input_variables=['input', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024E9AAB8D50>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Question: {input}' prefix='\nYou are an expert in converting natural language questions into MySQL queries. Your task is to generate a syntactically correct MySQL query based on the given input question, execute the query, and then provide an answer based on the query results.\nGuidelines:\n1. Construct a MySQL query to fetch data relevant to the user\'s question. Ensure the query is syntactically correct.\n2. If the question does not specify the number of examples, use the LIMIT clause to restrict the results to at most {top_k} entries. Prioritize 

In [28]:
# result = new_chain("What was my total earning last week as my account number is 409000493201'")

In [29]:
example_questions = [shot['Question'].strip() for shot in few_shots]
print(example_questions)

['Income of last 3 months. ?', 'How many transactions did I make in last 2 week as my account number is 409000493201?', 'Amount spent last week as my account number is 409000493201?', 'Expenses for each day this month ?', 'savings of previous year for each month', 'Amount Saved each year', 'Savings of last 2 year as my account number is 409000493201?', 'Expenses of each day of last week for account number 409000438611', 'Amount spent this year as my account number is 409000493201??', 'What is my consistent income in last 2 months as my account number is 409000493201  ?', 'amount/money saved each month previous year ?', 'What is my total income last quarter?', 'income of the previous year', 'income of the first 4 month of previous year', 'amount I lost each year?', 'What is my average spendings of last 11 transactions?', 'How much did I spend each quarter this year?', 'In which date did i make my highest transaction of this month and what was the amount?', 'How much amount did I spent o

In [30]:
from sentence_transformers import SentenceTransformer
# sentence-transformers/all-mpnet-base-v2
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
example_embeddings = model.encode(example_questions)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity  
# Function to determine if a user question is relevant to the database context
def is_question_relevant(user_question, similarity_threshold=0.40):
    user_embedding = model.encode([user_question])[0]  
    
    # calculating the cosine similarity between user question and example questions
    similarity_scores = cosine_similarity(user_embedding.reshape(1, -1), example_embeddings)[0]
    print(similarity_scores)
    
    max_similarity_score = max(similarity_scores)
    print(max_similarity_score)
    return max_similarity_score > similarity_threshold

In [33]:
from guardrails import Guard, OnFailAction
from guardrails.hub import CompetitorCheck, ToxicLanguage
from guardrails.hub import ProfanityFree

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\guardrails\validators\__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(


In [34]:

guard = Guard().use_many(
    CompetitorCheck(["khalti", "fusemachine","Deerhold","Deerwalk","Cotiviti"], on_fail=OnFailAction.EXCEPTION),
    ToxicLanguage(threshold=0.9, validation_method="sentence", on_fail=OnFailAction.EXCEPTION),
    ProfanityFree(on_fail=OnFailAction.EXCEPTION))


c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
# Function to handle user queries
def handle_user_query(user_question):
    if is_question_relevant(user_question):
        # response=new_chain(user_question)
        return True
    else:
        return "Your Question is not Relevant to the Context"

In [36]:
acc_no ="as my account number is 409000611074"

In [37]:
# I want to buy a car worth RS. 500000 in next 2 years . So how much should i save each month to buy the car  for account number 409000493201?
user_question = f"what was my savings last month ?  "
print(user_question)

what was my savings last month ?  


In [38]:
try:
    # Validate the user question
    guard.validate(user_question)  # If validation fails, an exception will be thrown
    # If validation passes, handle the user query
    response = handle_user_query(user_question)
except Exception as e:
    response = str(e)

[0.60203314 0.54559606 0.61633825 0.5932877  0.58444315 0.56527376
 0.60219204 0.4719022  0.5953816  0.5544629  0.64762926 0.65715337
 0.38640147 0.41634578 0.4967081  0.5363411  0.6930816  0.55434394
 0.78462815 0.6880093  0.53249955 0.667287   0.6415483  0.5532886
 0.6024281  0.6724714  0.5295236  0.57863104 0.5700383  0.46911293
 0.70733666 0.60420024 0.45137304 0.4374778  0.5718559  0.5586995
 0.58897936 0.59999925 0.67592335 0.5357217 ]
0.78462815


In [39]:
print(response)

True


In [40]:
example_selector.select_examples({f"Question":"what was my savings last month ? "})

[{'Answer': 'Your total income last month was RS. 4700000 and total spending was RS. 4610196 .',
  'Question': 'What was my total income and total spending last month? as my account number is 409000425051 ',
  'SQLQuery': "  SELECT SUM(Deposit_amount) AS Total_Income, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1 AND Account_No = '409000493201'; ",
  'SQLResult': "[(Decimal('4700000'), Decimal('4610196'))]"},
 {'Answer': 'You spent following amount :\n    07-01: RS. 116967,\n     07-02: RS. 90749 ',
  'Question': 'How much did I spend on the 1st and 2nd of this month for account number 409000493201?',
  'SQLQuery': "SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending \n                   FROM transactions \n                   WHERE YEAR(Value_date) = YEAR(CURDATE()) \n                   AND MONTH(Value_date) = MONTH(CURDATE()) \n                   AND DAY(V

In [41]:
if response==True:
    result=new_chain(user_question)
else:
    print(response)

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
what was my savings last month ?  
SQLQuery:SELECT Balance_amount - Value_amount AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'Value_amount' in 'field list'")
[SQL: SELECT Balance_amount - Value_amount AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
print(result)